In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import sys
sys.path.append("../")

import warnings
warnings.filterwarnings('ignore')

from kats.consts import TimeSeriesData

In [ ]:
multi_ts_df = pd.read_csv("../datasets/wit-10hz.csv", sep='\t')
multi_ts_df.columns = ["time", "acx", "acy", "acz","gyx", "gyy", "gyz", "roll", "pitch", "yaw"]
multi_ts_df.drop(["gyx", "gyy", "gyz", "roll", "pitch", "yaw"], axis=1, inplace=True)
multi_ts_df.head()

In [ ]:
multi_ts_df['timedf'] = pd.to_datetime(multi_ts_df['time'])
multi_ts_df.info()
multi_ts_df.drop('time', axis=1, inplace=True)
multi_ts_df.columns = ["acx", "acy", "acz", "time"]
multi_ts_df.head()

## Optional filtering

In [ ]:
multi_ts_df = multi_ts_df[(multi_ts_df['time'] > '2021-05-30 11:10') & (multi_ts_df['time'] < '2021-05-30 11:12')]
multi_ts_df = multi_ts_df[(multi_ts_df[["acx", "acy", "acz"]] > 0).all(1)]

In [ ]:
multi_ts = TimeSeriesData(multi_ts_df)

In [ ]:
# check that the type of the data is a "TimeSeriesData" object for both cases
print(type(multi_ts))

In [ ]:
# For the multi_ts TimeSeriesData, time is a pd.Series and value is a pd.DataFrame
print(type(multi_ts.time))
print(type(multi_ts.value))

In [ ]:
multi_ts_from_series = TimeSeriesData(time=multi_ts_df.time, value=multi_ts_df[['acx','acy','acz']])

In [ ]:
multi_ts == multi_ts_from_series

In [ ]:
# We can plot multiple time series from multi_ts by passing in the name of each value column we want to plot
multi_ts.plot(cols=['acx','acy','acz'])
plt.show()

In [ ]:
# Initiate feature extraction class
from kats.tsfeatures.tsfeatures import TsFeatures
tsFeatures = TsFeatures()

In [ ]:
features_ts = TsFeatures().transform(multi_ts)

In [ ]:
features_ts

## cmdstanpy sample

In [ ]:
import os
from cmdstanpy import cmdstan_path, CmdStanModel

# specify locations of Stan program file and data
stan_file = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.stan')
data_file = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.data.json')

# instantiate a model; compiles the Stan program by default
model = CmdStanModel(stan_file=stan_file)

# obtain a posterior sample from the model conditioned on the data
fit = model.sample(chains=4, data=data_file)

# summarize the results (wraps CmdStan `bin/stansummary`):
fit.summary()